In [12]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

In [13]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out


installations:

1. pip install synapseclient

In [14]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out


In [15]:
import os
import pandas as pd

full_fragments_and_cell_type_labels = [("/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118179_syn52119924_ENCSR052DKH.atac.filter.fragments.hg38.tsv.gz/ENCSR052DKH.atac.filter.fragments.hg38.tsv.gz",
                                        "/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/labels/syn34271785_ENCSR052DKH_syn46650146/cell_types.tsv.gz")]
local_clusters_fld = os.path.join(os.getcwd(),"clusters")
os.makedirs(local_clusters_fld, exist_ok=True)
local_path_to_download = os.path.join(os.getcwd(),"fragment_files")

In [16]:
def remove_file(filename):
  print("remove_file method: {}".format(filename))
  if os.path.exists(filename):
    os.remove(filename)

In [17]:
def save_clusters_mapping(fld,atac_dataset_id,list_to_save):
 # Create a DataFrame from the tuple list
    df = pd.DataFrame(list_to_save, columns=["ClusterId", "ManualAnnotationLabel"])

    # Specify the output TSV file path
    output_tsv_file = os.path.join(fld,atac_dataset_id,"{}_cluster_id_annotation_map.tsv".format(atac_dataset_id))
    print("sort_list_with_indices_and_save: output_tsv_file is {}".format(output_tsv_file))
    # Write the DataFrame to a TSV file
    df.to_csv(output_tsv_file, sep="\t", index=False)

In [7]:
# IMPORTNAT: we make sure to keep the same order if the cell_id and the cluster for the next execution
def sort_list_with_indices(input_list):
    sorted_list = sorted(input_list)
    indexed_sorted_list = [(i, item) for i, item in enumerate(sorted_list)]

    
    return indexed_sorted_list

In [ ]:
%run tsv_files_utils_from_analysis.ipynb

for local_file_tuple in full_fragments_and_cell_type_labels:

    # Dictionary to store the output file handles with names
    output_handles = {}
    local_fragment_file = local_file_tuple[0]
    full_cell_types_annotation_file_path = local_file_tuple[1]
    print("!!!!!local_fragment_file is {}".format(local_fragment_file))
    file_atac_dataset_id = local_fragment_file.split("_")[-1].split(".")[0]
    print("file_atac_dataset_id is {}".format(file_atac_dataset_id))
 
    # This full_cell_types_annotation_file_path was given as an input
    df_cell_types_for_atac_dataset = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_cell_types_annotation_file_path)
    cell_type_id_names_for_atac_dataset = list(set(df_cell_types_for_atac_dataset['cell_type_id'].to_list()))
    sorted_idx_cell_type_id_names_for_atac_dataset = sort_list_with_indices(cell_type_id_names_for_atac_dataset)
    print("sorted_idx_cell_type_id_names_for_atac_dataset is {}".format(sorted_idx_cell_type_id_names_for_atac_dataset))
    print("number of cell_type_id_names_for_atac_dataset is {}".format(len(sorted_idx_cell_type_id_names_for_atac_dataset)))
    # List of output text files with corresponding names
    # [("output1.txt", "file_1"), ("output2.txt", "file_2"), ("output3.txt", "file_3")]
    output_tagAlign_files_with_names = [(os.path.join(local_clusters_fld,file_atac_dataset_id,"{}_Cluster{}.tsv.tagAlign".format(file_atac_dataset_id,item[0])),
                                    item[1]) for item in sorted_idx_cell_type_id_names_for_atac_dataset]
    print("@@@@output_tagAlign_files_with_names is {}".format(output_tagAlign_files_with_names))

    # this will make sure that we will not run the same tagAlign twice.
    tagAlign_exists = [os.path.exists(output_tagAlign_file[0]) for output_tagAlign_file in output_tagAlign_files_with_names]
    print("tagAlign_exists is {}".format(tagAlign_exists))
    if sum(tagAlign_exists) >0:
        print("output_tagAlign_files_with_names {} is at work or was already downloaded. continue".format(output_tagAlign_files_with_names))
        continue # either started by annother process or already was processed
    else:
        print("!!!output_tagAlign_files_with_names {}. open files".format(output_tagAlign_files_with_names))
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            # print("tag_file_path is {}".format(tag_file_path))
            # print("os.path.dirname(tag_file_path) is {}".format(os.path.dirname(tag_file_path)))
            os.makedirs(os.path.dirname(tag_file_path), exist_ok=True)
            output_handles[tag_file_cell_type_name] = open(tag_file_path, "w")

    print("save the cluster to annotation mapping")
    save_clusters_mapping(local_clusters_fld,file_atac_dataset_id,sorted_idx_cell_type_id_names_for_atac_dataset)
    
    # print("!!!output_tagAlign_files_with_names {}".format(output_tagAlign_files_with_names))
    print("open local_fragment_file {}".format(local_fragment_file))
    with gzip.open(local_fragment_file, "rt") as infile:
        missing_bc = 0
        # Open the output files and store their handles in the list
        num_of_lines_written=0
        for line_number, line in enumerate(infile, start=1):
            # # debug
            # if line_number > 10:
            #     continue 

            # here cases where bc_datasetId or datasetId_bc are being mixed between the fragments and the
            # cell type are being address. you can select the righ out_list for your experiment
            out_list = split_fragment_line_string(line)
            bc = out_list[-1]
#             Austin output: chrom, start, end, bc, rem = line.rstrip('\n').split('\t', 5)
            out_line_to_print = f"{out_list[0]}\t{out_list[1]}\t{out_list[2]}\t{bc}\t{out_list[-2]}\n"
#             chr1	10007	10175	ENCSR023FME#ENCSR023FME_GAAGGTTCAAAGTGTCAGTCAA	1
            num_of_lines_written +=1
            returnTagAlign = convert_fragment_line_to_tagAlign(out_line_to_print)
            # write to the relevant cell type file
            # print("df_cell_types_for_atac_dataset[cell_id][0:5] is {}".format(df_cell_types_for_atac_dataset['cell_id'][0:5]))
            bc_exists_in_cell_type_atac_dataset = df_cell_types_for_atac_dataset[df_cell_types_for_atac_dataset['cell_id']==bc]
            # print("len(bc_exists_in_cell_type_atac_dataset) is {} for bc {}".format(len(bc_exists_in_cell_type_atac_dataset),bc))
            if len(bc_exists_in_cell_type_atac_dataset) ==1:
                tag_file_cell_type_id = df_cell_types_for_atac_dataset.loc[df_cell_types_for_atac_dataset['cell_id'] == bc, 'cell_type_id'].iloc[0]
                # print("tag_file_cell_type_id is {}".format(tag_file_cell_type_id))
                output_handles[tag_file_cell_type_id].write(returnTagAlign)
            else:
                missing_bc+=1
        
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            print("tag_file_path is {}".format(tag_file_path))
            output_handles[tag_file_cell_type_name].close()    
        print("finished clustering local_fragment_file {} by cell type. for types {}".format(local_fragment_file, cell_type_id_names_for_atac_dataset))
        print("total missing bc are {}".format(missing_bc))

!!!!!local_fragment_file is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118179_syn52119924_ENCSR052DKH.atac.filter.fragments.hg38.tsv.gz/ENCSR052DKH.atac.filter.fragments.hg38.tsv.gz
file_atac_dataset_id is ENCSR052DKH
tsv_gz_file is /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/labels/syn34271785_ENCSR052DKH_syn46650146/cell_types.tsv.gz
sorted_idx_cell_type_id_names_for_atac_dataset is [(0, 'Macrophage'), (1, 'T_lymphocyte'), (2, 'Tuft_cell'), (3, 'Vascular_smooth_muscle_cell')]
number of cell_type_id_names_for_atac_dataset is 4
@@@@output_tagAlign_files_with_names is [('/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/ENCSR052DKH/ENCSR052DKH_Cluster0.tsv.tagAlign', 'Macrophage'), ('/Users/eilaari

In [9]:
# sum([True, False, False, False])

In [10]:
sorted_idx_cell_type_id_names_for_atac_dataset

[(0, 'Macrophage'),
 (1, 'T_lymphocyte'),
 (2, 'Tuft_cell'),
 (3, 'Vascular_smooth_muscle_cell')]

In [11]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/


